In [0]:
import os
import numpy as np
import pandas as pd


In [0]:
import pickle

In [2]:
os.getcwd()

'/content'

In [0]:
dataset=pd.read_csv('train.csv')

In [4]:
dataset.shape


(5000000, 3)

In [0]:
def batchnum(ts,div):
  qt=(float(ts)*10000)//div
  if ((float(ts)*10000)%div)==0:
    return 'b'+str(int(qt))
  else:
    return 'b'+str(int(qt+1))

In [0]:
dataset['batch_discrete']=dataset.time.apply(lambda x : batchnum(x,500000))

In [0]:
dataset['batch_sec']=dataset.time.apply(lambda x : batchnum(x,10000))

In [0]:
dataset['batch_10khz']=dataset.time.apply(lambda x : batchnum(x,500))

In [9]:
dataset.batch_discrete.unique()

array(['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10'],
      dtype=object)

In [0]:
#dataset.batch_sec.unique()

In [102]:
dataset.head(2)

,time,signal,open_channels,batch_discrete,batch_sec,batch_10khz
0,0.0001,-2.7600,0,b1,b1,b1
1,0.0002,-2.8557,0,b1,b1,b1


In [0]:
for col in ['batch_discrete','batch_sec','batch_10khz']:
    dct={}
    dct[col+'mean']=dataset[['signal',col]].groupby(col)['signal'].mean()
    dct[col+'median']=dataset[['signal',col]].groupby(col)['signal'].median()
    dct[col+'stdev']=dataset[['signal',col]].groupby(col)['signal'].std()
    dct[col+'min']=dataset[['signal',col]].groupby(col)['signal'].min()
    dct[col+'max']=dataset[['signal',col]].groupby(col)['signal'].max()
    dct[col+'25percentile']=dataset[['signal',col]].groupby(col)['signal'].quantile(.25)
    dct[col+'75percentile']=dataset[['signal',col]].groupby(col)['signal'].quantile(.75)
    for val in dct:
            dataset[val] = dataset[col].map(dct[val].to_dict())
    dataset[col+'range']=dataset[col+'max']-dataset[col+'min']
    dataset[col+'minmax']=dataset[col+'min']/dataset[col+'max']
    dataset[col+'prev']=dataset['signal'].shift(-1).fillna(dataset['signal'].values[-2])
    dataset[col+'next']=dataset['signal'].shift(+1).fillna(dataset['signal'].values[1])

In [11]:
dataset.head(2)
len(dataset.columns)

39

In [0]:
X=dataset.drop(columns=['time','batch_discrete','batch_sec','batch_10khz','signal','open_channels']).values

In [0]:
y=dataset[['open_channels']].values

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


In [0]:
from sklearn.linear_model import LogisticRegression
logmodel=LogisticRegression()

In [17]:
logmodel.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predictions = logmodel.predict(X_test)

In [21]:
from sklearn.metrics import classification_report
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.95      0.93      0.94    310144\n           1       0.84      0.81      0.83    246264\n           2       0.61      0.65      0.63    138257\n           3       0.63      0.55      0.59    167164\n           4       0.45      0.81      0.58    100448\n           5       0.06      0.01      0.01     69833\n           6       0.37      0.37      0.37     46926\n           7       0.43      0.42      0.43     66622\n           8       0.43      0.64      0.52     61240\n           9       0.47      0.26      0.34     34006\n          10       0.19      0.20      0.19      9096\n\n    accuracy                           0.68   1250000\n   macro avg       0.49      0.51      0.49   1250000\nweighted avg       0.67      0.68      0.66   1250000\n'

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[287670,  20888,   1520,     46,     14,      0,      0,      0,
             0,      3,      3],
       [ 14481, 200613,  28442,   1933,    685,      1,     24,      0,
             2,     36,     47],
       [   727,   8038,  89826,  28849,  10344,      3,    142,      1,
            10,    128,    189],
       [   177,   1600,  22067,  92328,  49440,     51,    942,     24,
            51,    190,    294],
       [    79,   1576,   1142,  10111,  80986,    272,   4649,    367,
           115,    434,    717],
       [    39,   2576,   1437,   2426,  38238,    360,  13359,   4009,
           483,   1323,   5583],
       [     8,   2100,   1407,   2911,     73,    734,  17174,  17931,
          4557,     31,      0],
       [     1,    899,    531,   4293,     48,   1954,   8351,  27899,
         22192,    433,     21],
       [     0,    142,    105,   2659,     26,   1888,   1637,  12287,
         38998,   3286,    212],
       [     0,     13,     11,    669,      6,    486,

In [19]:
#dataset.head(1)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.67736

# New Section